---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [3]:
len(X_train)

4179

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [4]:
def answer_one():
    
    
    return sum(spam_data['target']/len(spam_data))*100 #Your answer here

In [5]:
answer_one()

13.406317300789542

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    a = vect.get_feature_names()
    sorted_a = sorted(a, key=len, reverse=True)
    sorted_a[0]
    
    return sorted_a[0] #Your answer here

In [7]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    classifier = MultinomialNB(alpha=0.1)
    classifier.fit(X_train_vect, y_train)
    predict = classifier.predict(vect.transform(X_test))
    auc = roc_auc_score(y_test, predict)
    
    
    return auc #Your answer here

In [9]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vec = vect.transform(X_train)
    tfidf_names = np.array(vect.get_feature_names())
    sort_tfidf = X_train_vec.max(0).toarray()[0].argsort()
    t_df = pd.DataFrame(X_train_vec.toarray(),  columns = vect.get_feature_names())
    b =t_df.T
    b['max'] = b.max(axis=1)
    b = b.sort_values(by='max')
    mins = b.iloc[0:20]
    mins = mins.sort_index().sort_values(by='max', kind='mergesort')
    mins = mins['max']

    maxs = b.iloc[-20:]
    maxs = maxs.sort_index().sort_values(by='max', kind='mergesort', ascending=False)
    maxs = maxs['max']
    mins.name=None
    mins.index.name=None
    maxs.name=None
    maxs.index.name=None
    
    return mins, maxs #Your answer here

answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dt

### q4 notes
need to transpose this df then find the max idf accross documents. there are ~4k documents and ~7k idf features

https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/4y22x-3KEeeaYw6MXNSBHA
https://stackoverflow.com/questions/34449127/sklearn-tfidf-transformer-how-to-get-tf-idf-values-of-given-words-in-documen
'''




### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [34]:
def answer_five():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import naive_bayes
    from sklearn.metrics import roc_auc_score
    min_df=3
    
    vect=TfidfVectorizer(min_df=min_df).fit(X_train)
    X_train_vect=vect.transform(X_train)
    
    vect_test=TfidfVectorizer(min_df=min_df).fit(X_train)
    X_test_vect=vect_test.transform(X_test)
    
    #build model
    clfrNB=naive_bayes.MultinomialNB(alpha=0.1)
    clfrNB.fit(X_train_vect, y_train)
    predictions=clfrNB.predict(X_test_vect)
    
    
    
    return roc_auc_score(y_test, predictions) #Your answer here

In [36]:
answer_five()

0.9416243654822335

### q5 notes

https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/cLJUR33kEeeUgw7JoFNoqA

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [37]:
def answer_six():
    #find len of text
    spam_data['len']=spam_data['text'].apply(len)
    
    #mean len of all messages
    all_len=spam_data['len'].mean()
    
    #mean len of not spam
    not_spam=spam_data[spam_data['target']==0]
    n_spam_len=not_spam['len'].mean()

    #mean len of spam
    is_spam=spam_data[spam_data['target']==1]
    is_spam_len=is_spam['len'].mean()

    
    return n_spam_len, is_spam_len#Your answer here

In [38]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [39]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [40]:
# #WORKING
# n=500
# X_train = X_train[:n]
# X_test=X_test[:n]
# y_train=y_train[:n]
# y_test=y_test[:n] 

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn import svm
# from sklearn.metrics import roc_auc_score
    

# min_df=5

# vect = TfidfVectorizer(min_df=5).fit(X_train)

# X_train_transformed = vect.transform(X_train)
# X_test_transformed= vect.transform(X_test)

# #add feature to vectorized data
#     #add feature x train
# X_train_length = X_train.str.len()
# X_train_transformed_added=add_feature(X_train_transformed, X_train_length)
#     #vect test
# X_test_length = X_test.str.len()
# X_test_transformed_added=add_feature(X_test_transformed, X_test_length)

# #fit a Support Vector Classification model with regularization C=10,000
# clfrSVM = svm.SVC(kernel='linear', C=10000)
# clfrSVM.fit(X_train_transformed_added, y_train)
# predictions=clfrSVM.predict(X_test_transformed_added)
# score=roc_auc_score(y_test, predictions)



In [17]:
# print('X_train_transformed', X_train_transformed.shape)
# print('X_train_transformed_added', X_train_transformed_added.shape)
# print('X_train_length', X_train_length.shape)
# print('X_test_length', X_test_length.shape)
# print('X_test_transformed_added', X_test_transformed_added.shape)
# score
# type(score)

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [41]:
def answer_seven():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import svm
    from sklearn.metrics import roc_auc_score


    min_df=5

    vect = TfidfVectorizer(min_df=5).fit(X_train)

    X_train_transformed = vect.transform(X_train)
    X_test_transformed= vect.transform(X_test)

    #add feature to vectorized data
        #add feature x train
    X_train_length = X_train.str.len()
    X_train_transformed_added=add_feature(X_train_transformed, X_train_length)
        #vect test
    X_test_length = X_test.str.len()
    X_test_transformed_added=add_feature(X_test_transformed, X_test_length)

    #fit a Support Vector Classification model with regularization C=10,000
    clfrSVM = svm.SVC(C=10000)
    clfrSVM.fit(X_train_transformed_added, y_train)
    predictions=clfrSVM.predict(X_test_transformed_added)
    score=roc_auc_score(y_test, predictions)

    
    return score

In [42]:
answer_seven()


0.9661689557407943

In [20]:
# y_train.shape

In [21]:
# X_len = X_train.apply(len)
# num_chars = X_train.str.len()
# #X_Tlen=add_feature(X_train, X_len)
# X_train.shape, X_len.shape

In [22]:
# print(X_len.describe())
# print(num_chars.describe())

### RE Notes

https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/LERUF8t_EeeTYAoJO24o4g

https://www.coursera.org/learn/python-text-mining/discussions/forums/RBU1oGgSEeePcg7f_RfZAA/threads/ZXn6vrSsSRy5-r60rFkcDg


https://www.coursera.org/learn/python-text-mining/discussions/forums/RBU1oGgSEeePcg7f_RfZAA/threads/33lKzvvlEeenrw7feJxQmg

https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/alpIN9MMEeihvg7K_fsdxA

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [43]:
def answer_eight():
    #https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/pFIa5tElEemOLBJyhsIcKA
    spam_data['Digit Count']=spam_data['text'].str.count(r'\d')
    is_spam=spam_data[spam_data['target']==1]
    not_spam=spam_data[spam_data['target']!=1]
    
    a=is_spam['Digit Count'].mean()
    b=not_spam['Digit Count'].mean()
    
    return b, a#Your answer here

In [44]:
answer_eight()


(0.2992746113989637, 15.759036144578314)

In [45]:
spam_data['Digit Count']=spam_data['text'].str.count(r'\d')
spam_data.head(20)
#spam_data['text'].iloc[9]

,text,target,len,Digit Count
0,"Go until jurong point, crazy.. Available only ...",0,111,0
1,Ok lar... Joking wif u oni...,0,29,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,25
3,U dun say so early hor... U c already then say...,0,49,0
4,"Nah I don't think he goes to usf, he lives aro...",0,61,0
5,FreeMsg Hey there darling it's been 3 week's n...,1,148,4
6,Even my brother is not like to speak with me. ...,0,77,0
7,As per your request 'Melle Melle (Oru Minnamin...,0,160,1
8,WINNER!! As a valued network customer you have...,1,158,19
9,Had your mobile 11 months or more? U R entitle...,1,154,13


### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [46]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import svm
    from sklearn.metrics import roc_auc_score


    min_df=5

    vect = TfidfVectorizer(min_df=min_df, ngram_range=(1,3)).fit(X_train)

    X_train_transformed = vect.transform(X_train)
    X_test_transformed= vect.transform(X_test)    
    
    #add [length of document and number of digits] features to vectorized data
        #add features x train
    X_train_length = X_train.str.len()
    X_train_cdigits=X_train.str.count(r'\d')
    X_train_transformed_added=add_feature(X_train_transformed, [X_train_length,X_train_cdigits])
        #add features vect test
    X_test_length = X_test.str.len()
    X_test_cdigits=X_test.str.count(r'\d')
    X_test_transformed_added=add_feature(X_test_transformed, [X_test_length, X_test_cdigits])
    
    #fit a logistic regression model with regularization C=100
    l_model = LogisticRegression(C=100)
    l_model.fit(X_train_transformed_added, y_train)
    predictions=l_model.predict(X_test_transformed_added)
    score=roc_auc_score(y_test, predictions)
    
    return  score #Your answer here

In [47]:
a=answer_nine()
a

C:\Users\reson\anaconda3\envs\working_env\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9809793219360643

In [28]:
# a = [1,2]
# type(a)

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [48]:
def answer_ten():
    import re
    
    spam_data['Non-word Count']=spam_data['text'].str.count(r'\W')
    is_spam=spam_data[spam_data['target']==1]
    not_spam=spam_data[spam_data['target']!=1]
    
    a=is_spam['Non-word Count'].mean()
    b=not_spam['Non-word Count'].mean()
    
    return b,a #Your answer here

In [49]:
a=answer_ten()
a

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [50]:
def answer_eleven():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import svm
    from sklearn.metrics import roc_auc_score
    import re


    min_df=5

    vect = CountVectorizer(min_df=min_df, ngram_range=(2,5), analyzer='char_wb').fit(X_train)

    X_train_transformed = vect.transform(X_train)
    X_test_transformed= vect.transform(X_test)    
    
    #add [len record, num digits, count non-word charecters] features to vectorized data
        #add features x train
    length_of_doc = X_train.str.len()
    digit_count = X_train.str.count(r'\d')
    non_word_char_count = X_train.str.count(r'\W')
    X_train_transformed_added=add_feature(X_train_transformed, [length_of_doc, digit_count, non_word_char_count])
        #add features vect test
    length_of_doc = X_test.str.len()
    digit_count = X_test.str.count(r'\d')
    non_word_char_count = X_test.str.count(r'\W')
    X_test_transformed_added=add_feature(X_test_transformed, [length_of_doc, digit_count, non_word_char_count])
    
    #fit a logistic regression model with regularization C=100
    l_model = LogisticRegression(C=100)
    l_model.fit(X_train_transformed_added, y_train)
    predictions=l_model.predict(X_test_transformed_added)
    score=roc_auc_score(y_test, predictions)
    
    
    #get smallest and largest coefficients
    feature_names= np.array(vect.get_feature_names()).tolist()
    
    new_fs=['length_of_doc', 'digit_count', 'non_word_char_count']
    for x in new_fs:feature_names.append(x)
    
    feature_names=np.array(feature_names)
    
    sorted_tfidf_index = X_train_transformed_added.max(0).toarray()[0].argsort()
    
    smallest_cs=feature_names[sorted_tfidf_index[:10]].tolist()
    largest_cs=feature_names[sorted_tfidf_index[:-11:-1]].tolist()
    
    
    return score, smallest_cs, largest_cs #Your answer here

In [32]:
score, smallest_cs, largest_cs =answer_eleven()
score, smallest_cs, largest_cs

C:\Users\reson\anaconda3\envs\working_env\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9813973821367333,
 ['getz', 'jay ', 'jea', 'jec', 'ject', 'ject ', 'jes', 'ji', 'jit', 'job'],
 ['length_of_doc',
  'non_word_char_count',
  'digit_count',
  '. ',
  't;',
  ' . ',
  ' .',
  't ',
  ' t',
  '..'])

https://www.coursera.org/learn/python-text-mining/discussions/weeks/3/threads/89fce4aCEee3oBJxXb3ZyA

change 3 new feature names and add them to the list of feature names
     ['length_of_doc', 'digit_count', 'non_word_char_count']